In [1]:
import os
os.environ

environ{'SHELL': '/bin/bash',
        'NUGET_XMLDOC_MODE': 'skip',
        'COLORTERM': 'truecolor',
        'CLOUDENV_ENVIRONMENT_ID': 'c73279f9-183f-42a8-b9cc-4abc8294d22f',
        'NVM_INC': '/usr/local/share/nvm/versions/node/v20.19.0/include/node',
        'TERM_PROGRAM_VERSION': '1.100.3',
        'GITHUB_USER': 'blahblahradio',
        'rvm_prefix': '/usr/local',
        'CODESPACE_NAME': 'jubilant-doodle-r99xqqv7qrq3x94q',
        'HOSTNAME': 'codespaces-659b4e',
        'JAVA_ROOT': '/home/codespace/java',
        'JAVA_HOME': '/usr/local/sdkman/candidates/java/current',
        'DOTNET_ROOT': '/usr/share/dotnet',
        'CODESPACES': 'true',
        'PYTHON_ROOT': '/home/codespace/.python',
        'GRADLE_HOME': '/usr/local/sdkman/candidates/gradle/current',
        'NVS_DIR': '/usr/local/nvs',
        'NVS_OS': 'linux',
        'DOTNET_SKIP_FIRST_TIME_EXPERIENCE': '1',
        'MY_RUBY_HOME': '/usr/local/rvm/rubies/ruby-3.3.4',
        'OPENAI_API_KEY': 'sk-proj-ce1Klz3lN